In [33]:
import pandas as pd
import numpy as np
import os

import PivotTable as pt
import process_mangement as pm

import warnings
warnings.filterwarnings("ignore")

In [34]:
# readDb = pm.readDb(db_path)

# df_attrs_stdds = readDb.read_table('표준데이터시트_개별속성')
# df_attrs_nonstdds = readDb.read_table('비표준데이터시트_개별속성')

# df_attrs_header = readDb.read_table('속성해더')

In [35]:
# 실제 코드가 완성된 후에 앞부분은 SQLite에서 QUERY를 통해 가져와서 바로 DATAFRAME으로 변환하는 방식으로 변경할 예정
# 지금은 테스트를 위해 CSV 파일을 사용

# 데이터 불러오기
base_path_test = os.path.join(os.getcwd(), 'test_file')

df_working_f = pd.read_csv(os.path.join(base_path_test, '개별속성_작업파일(Test).csv'), dtype={'공정번호' : str}) # 작업파일
df_attrs_stdds = pd.read_csv(os.path.join(base_path_test, '개별속성_표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 표준데이터시트
df_attrs_nonstdds = pd.read_csv(os.path.join(base_path_test, '개별속성_비표준데이터시트(Test).csv'), dtype={'공정번호' : str}) # 비표준데이터시트
df_attrs_header = pd.read_csv(os.path.join(base_path_test, '속성해더.csv'), dtype={'공정번호' : str}) # 속성해더

In [36]:
df_attrs_header = df_attrs_header[df_attrs_header['속성 그룹 코드']=='01_속성명']

df_attrs_list = {'표준데이터시트' : df_attrs_stdds, '선작업데이터' : df_attrs_nonstdds}

df_fins = []

for source, df_attrs in df_attrs_list.items() :
    df_attrs_header = df_attrs_header[df_attrs_header['속성 그룹 코드']=='01_속성명']
    insert_attrs_preprocessing = pm.InsertAttrstPreprocessing(df_working_f, df_attrs_header)
    insert_attrs_preprocessing.step0_1()
    insert_attrs_preprocessing.step0_2(df_attrs, col_name=source)

    df_working_f = insert_attrs_preprocessing.df_working_f
    df_attrs = insert_attrs_preprocessing.df_attrs

    df_fin = pd.DataFrame(columns=df_attrs_header.columns)
    df_results =[]

    for i in range(len(df_attrs_header)) :
        insert_attrs_pipeline = pm.InsertAttrsPipeline(df_working_f, df_attrs, df_attrs_header)
        df_result = insert_attrs_pipeline.excute(i)
        df_results.append(insert_attrs_pipeline.df_cct_result)

    df_results.insert(0, df_fin)
    df_fin = pd.concat(df_results, axis=0)

    df_fin = pd.concat([df_fin, df_working_f], ignore_index=True)
    df_fin = df_fin.drop_duplicates(subset=['SRNo', '공종별 분류 코드'], keep='first')
    df_fin.sort_values(by=['공종별 분류 코드', '속성 그룹 코드'], inplace=True)

    df_fins.append(df_fin)

In [37]:
def merge_and_update(df1, df2) :
    """우선순위 업데이트(특정 칼럼을 키 값으로 할 경우)"""
    df_merged = pd.merge(df1, df2, on=['SRNo', '공종별 분류 코드'], suffixes=['_1st', '_2nd'], how='left')

    # update할 칼럼에 대하여 combine_first 수행
    update_columns = [f'속성{i}' for i in range(1, 8)]
    for col in update_columns :
        df_merged[col] = df_merged[f"{col}_1st"].combine_frist(df_merged[f'{col}_2nd'])

    return df_merged

In [39]:
df_merged = merge_and_update(df_fin[0], df_fin[1])
df_merged

KeyError: 0

In [28]:
df_fins[0]

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,공종별 분류 코드,속성 그룹 코드,속성1,속성2,속성3,속성4,속성5,속성6,속성7
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,AMB,NaN,20|m^3/h,유,2000,No,3|mm
2,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,45,F.V,NaN,Yes,1000,NaN,NaN
9,AE00001,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,AE00004,NaN,PID,07,LS0002,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,AE00005,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,11001,01_속성명,서비스,타입,측정범위,제조사,정비주기,NaN,NaN
4,AE0006,NaN,PID,07,NaN,HH0003,11001,03_DATA,Tk-503,Bourdon,range 0 ~ 50,영월압력계,1년,NaN,NaN
5,AE0007,NaN,PID,07,NaN,HH0004,11001,03_DATA,NaN,Bourdon,NaN,NaN,NaN,NaN,NaN
17,AE0008,NaN,PID,07,NaN,NaN,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_fins[1]

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,공종별 분류 코드,속성 그룹 코드,속성1,속성2,속성3,속성4,속성5,속성6,속성7
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,15(비표준),비표준에서 들어옴,20|m^3/h(비표준),유(비표준),2000(비표준),Yes(비표준),3|mm
2,AE00004,NaN,PID,07,LS0002,NaN,10001,03_DATA,50(비표준),F.V(비표준),50(비표준),NaN,1000(비표준),NaN,NaN
7,AE00001,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,AE00005,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,11001,01_속성명,서비스,타입,측정범위,제조사,정비주기,NaN,NaN
13,AE0006,NaN,PID,07,NaN,HH0003,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,AE0007,NaN,PID,07,NaN,HH0004,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,AE0008,NaN,PID,07,NaN,NaN,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
update_columns = [f'속성{i}' for i in range(1, 8)]
df_working_f_result = df_working_f.copy()

df_working_f_result[update_columns] = df_fins[0][update_columns].combine_first(df_fins[1][update_columns])
df_working_f_result

,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,공종별 분류 코드,속성 그룹 코드,속성1,속성2,속성3,속성4,속성5,속성6,속성7
0,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,10001,01_속성명,온도,압력,유량,방폭,크기,비파괴,부식여유
1,AE00001,NaN,PID,07,NaN,NaN,10001,03_DATA,AMB,비표준에서 들어옴,20|m^3/h,유,2000,No,3|mm
2,AE00002,NaN,PID,07,LS0001,HH0001,10001,03_DATA,45,F.V,50(비표준),Yes,1000,NaN,NaN
3,HH0002,NaN,표준,07,NaN,HH0002,10001,03_DATA,서비스,타입,측정범위,제조사,정비주기,NaN,NaN
4,AE00004,NaN,PID,07,LS0002,NaN,10001,03_DATA,Tk-503,Bourdon,range 0 ~ 50,영월압력계,1년,타입,NaN
5,AE00005,NaN,PID,07,NaN,NaN,10001,03_DATA,NaN,Bourdon,짱구(비표준),Bourdon(비표준),NaN,NaN,NaN
6,SRNo,작업자,출처,공정번호,선작업데이터,표준데이터시트,11001,01_속성명,온도,압력,유량,비파괴,제조사,타입,NaN
7,AE0006,NaN,PID,07,NaN,HH0003,11001,03_DATA,15|degC,1.5|kgf/cm^2,NaN,No,놀이공원안전사고현황,의자사용방법,NaN
8,AE0007,NaN,PID,07,NaN,HH0004,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AE0008,NaN,PID,07,NaN,NaN,11001,03_DATA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_working_f_result.to_csv(os.path.join(base_path_test, '개별속성_작업파일_결과(Test)-result.csv'), index=False, encoding='cp949')